In [25]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from time import sleep
import random
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [26]:
places_raw = pd.read_excel('places.xlsx', header=None)
places = places_raw[0].values

In [2]:
from bs4 import BeautifulSoup
import requests
u=list()
country_code="us"
url="https://www.proxynova.com/proxy-server-list/country-"+country_code+"/"
soup = BeautifulSoup(requests.get(url).text,"html.parser")
allproxy = soup.find("tbody").find_all("tr")
count = 0
for proxy in allproxy:
    print(count)
    count+=1
    foo = proxy.find_all("td")   
    print(foo)
    try:
        l={}
        l["ip"]=foo[0].find("script").contents[0].replace("\n","").replace(")","")
        l["ip"]=l["ip"].replace("document.write(","").replace("\’","").replace(";","").replace("\"","").replace(" + ","")
        print(l['ip'])
        l["port"]=foo[1].contents[0].replace("\n","").replace(" ","")
        l["country"]=foo[5].find("a").text.replace("\n","").replace(" ","")
        u.append(l)
    except Exception as ex:
        print(ex)
print(u)


0
[<td align="left">
<script>document.write("20." + "47." + "108" + ".20" + "4");</script>
</td>, <td align="left">
                                    8888
                            </td>, <td align="left">
<time class="icon icon-check timeago" datetime="2022-04-21 14:33:54Z"></time>
</td>, <td align="left">
<div class="progress-bar" data-value="0" title="158"></div>
<small>158 ms</small>
</td>, <td class="text-center text-sm">
<span class="uptime-medium">40%</span>
<span> (609)</span>
</td>, <td align="left">
<img alt="us" class="flag flag-us inline-block align-middle" src="/assets/images/blank.gif"/>
<a href="/proxy-server-list/country-us/" title="Proxies from United States">United States

                                            <span class="proxy-city"> - Washington </span>
</a>
</td>, <td align="left">
<span class="proxy_elite font-weight-bold smallish">Elite</span>
</td>]
20.47.108.204
1
[<td align="left">
<script>document.write("64." + "227" + ".62" + ".12" + "3");</script

In [4]:
#ip_addresses = ["85.237.57.198:44959", "116.0.2.94:43379", "186.86.247.169:39168"]
#ip_addresses = [i['ip']+':'+str(i['port']) for i in u]
#print(ip_addresses)
for place in places:
    try:
        #proxy_index = random.randint(0, len(ip_addresses) - 1)
        #proxies = {"http": "http://"+ip_addresses[proxy_index], "https": "https://"+ip_addresses[proxy_index]}
        url = "https://"+place+".craigslist.org/search/sss?query=bikes"
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}

        from requests.adapters import HTTPAdapter
        from requests.packages.urllib3.util.retry import Retry
        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)

        response = session.get(url)
        #response = requests.get(url, headers=headers,proxies = proxies)
        if response.status_code == 200:
            print("Success")
        else:
            print("Failure")
            #requests.get(url, proxies=proxies)
    except Exception as ex:
        # implement here what to do when there's a connection error
        # example: remove used proxy from  pool and retry request using another one
        print("failure")
        print(ex)

Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success


KeyboardInterrupt: 

In [14]:
url = "https://newyork.craigslist.org/search/sss?query=bikes"
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
if response.status_code == 200:
    print("Success")
else:
    print("Failure")

Success


In [30]:
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from tqdm import tqdm

prices = []
urls = []
dates = []
locations = []
titles = []

for place in tqdm(places):
    
    try:
        #proxy_index = random.randint(0, len(ip_addresses) - 1)
        #proxies = {"http": "http://"+ip_addresses[proxy_index], "https": "https://"+ip_addresses[proxy_index]}
        url = "https://"+place+".craigslist.org/search/sss?query=bikes"
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}

        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)

        response = session.get(url)
        #response = requests.get(url, headers=headers,proxies = proxies)
        if response.status_code != 200:
            print("Failure")

            #requests.get(url, proxies=proxies)
    except Exception as ex:
        # implement here what to do when there's a connection error
        # example: remove used proxy from  pool and retry request using another one
        print("failure")
        print(ex)

    results_page = BeautifulSoup(response.content, 'lxml')

    # find the total number of posts
    results_num = results_page.find('div', class_='search-legend')
    results_total = int(results_num.find('span', class_='totalcount').text)

    pages = np.arange(0, results_total+1, 120)

    # Loop through all pages
    count = 0
    for page in pages:
        
        try:
            session = requests.Session()
            retry = Retry(connect=3, backoff_factor=0.5)
            adapter = HTTPAdapter(max_retries=retry)
            session.mount('http://', adapter)
            session.mount('https://', adapter)

            response = session.get(url+ "s="  # the parameter for defining the page number
                                    # the page number in the pages array from earlier
                                    + str(page)
                                    + "&hasPic=1"
                                    + "&availabilityMode=0")

            if response.status_code != 200:
                print("Failure")

            #requests.get(url, proxies=proxies)
        except:
            # implement here what to do when there's a connection error
            # example: remove used proxy from  pool and retry request using another one
            
            print("Failure "+str(count))
            count+=1
            pass

        sleep(0.5)
        if response.status_code != 200:
            print(('Request: {}; Status code: {}'.format(
                requests, response.status_code)))

        page_html = BeautifulSoup(response.text, 'html.parser')

        posts = results_page.find_all('li', class_='result-row')

        for post in posts:
            try:
                prices.append(int(post.find('span', class_="result-price").text[1:].replace(',', '')))
                urls.append(post.find('a', class_="result-title hdrlnk")['href'])
                dates.append(post.find('time', class_="result-date")['datetime'])
                locations.append(place)
                titles.append(post.find('a', class_="result-title hdrlnk").text)
            except:
                pass

100%|██████████| 417/417 [25:13<00:00,  3.63s/it] 


In [45]:
df = pd.DataFrame({'Price':prices, 'Location':locations, 'URL':urls, 'Date':dates, 'Title':titles})
df

,Price,Location,URL,Date,Title
0,1000,auburn,https://auburn.craigslist.org/mcd/d/fayettevil...,2022-04-05 15:32,"BAD CREDIT, NO CREDIT, OK! WE WORK WITH EVERYONE!"
1,1000,auburn,https://auburn.craigslist.org/mcd/d/fayettevil...,2022-03-21 13:55,"BAD CREDIT, NO CREDIT, OK! WE WORK WITH EVERYONE!"
2,0,auburn,https://auburn.craigslist.org/wad/d/auburn-wan...,2022-04-17 22:46,Wanted Old Motorcycles 📞1(800) 220-9683 www.wa...
3,1,auburn,https://auburn.craigslist.org/for/d/defuniak-s...,2022-04-17 13:32,Atv’s
4,8795,auburn,https://auburn.craigslist.org/fod/d/fairburn-2...,2022-04-16 14:51,2017 Club Car Precedent 4 Seater Gas Alabama C...
...,...,...,...,...,...
168282,3200,wyoming,https://fortcollins.craigslist.org/bik/d/fort-...,2022-04-15 13:43,Black Sheep Pine Ridge SS titanium Cyclocross ...
168283,900,wyoming,https://fortcollins.craigslist.org/bik/d/fort-...,2022-04-15 13:43,Specialized S-works Epic Mountain Bike
168284,14995,wyoming,https://fortcollins.craigslist.org/mcd/d/scott...,2022-04-15 13:28,2011 Harley Davidson Ultra Classic with Trike
168285,112,wyoming,https://logan.craigslist.org/sgd/d/full-garage...,2022-04-15 12:36,"Full Garage Gym-Squat Rack, Dumbbells- Financi..."


In [46]:
df = df.where(df['Price']!=0).dropna()
df = df.drop_duplicates(subset=['Price', 'Location', 'Title']).reset_index(drop=True)

In [47]:
df.to_csv('dataurls.csv')

In [18]:
import requests
from lxml.html import fromstring
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return list(proxies)

proxies = get_proxies()
print(proxies)

['63.161.104.189:3128']


In [27]:
df = pd.read_csv('dataurls.csv').drop(columns=['Unnamed: 0'])

In [28]:
from tqdm import tqdm 
from os.path import exists

def scrape(df, city):

    df = df.where(df['Location']==city).dropna()

    for ind in tqdm(df.index):

        url_ = df.loc[ind, 'URL']

        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=1)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)

        response = session.get(url_, headers={'User-Agent': 'Mozilla/5.0'})

        if response.status_code != 200:
            print(('Request: {}; Status code: {}'.format(requests, response.status_code)))

        else:
            page_html = BeautifulSoup(response.text, 'html.parser')
            
            all_attributes = page_html.find('p', class_= 'attrgroup')

            # Finds all the attributes
            try:
                attr = all_attributes.find_all('span')

                for att in attr:
                    str_attr = att.text.split(':')
                    attribute, value = str_attr
                    if attribute not in df.columns:
                        df[attribute] = [np.nan for _ in range(len(df))]
                    df.loc[ind, attribute] = value
            except:
                pass

            # Finds the number of images
            try:
                user_body = page_html.find('section', class_="userbody")
                multi_img = user_body.find('figure', class_="iw multiimage")
                num_img = multi_img.find('span', class_="slider-info").text.split(' ')[-1]
                if 'num image' not in df.columns:
                    df['num image'] = [0 for _ in range(len(df))]
                df.loc[ind, 'num image'] = num_img
            except:
                if 'num image' not in df.columns:
                    df['num image'] = [0 for _ in range(len(df))]
                df.loc[ind, 'num image'] = 0

            # Find the text
            try:
                user_text = str(page_html.find('section', {'id':"postingbody"}).text).strip().replace('QR Code Link to This Post', '').replace('\n', ' ').strip()
                if 'text' not in df.columns:
                        df['text'] = [np.nan for _ in range(len(df))]
                df.loc[ind, 'text'] = user_text
            except:
                pass
    
    df.to_csv('data_craigslist'+str(city)+'.csv')

for city in places:
    if not exists('data_craigslist'+str(city)+'.csv'):
        scrape(df, city)
        sleep(2)

  6%|▋         | 7/112 [00:01<00:16,  6.50it/s]

Request: <module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>; Status code: 403


 11%|█         | 12/112 [00:01<00:16,  6.14it/s]

Request: <module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>; Status code: 403


 17%|█▋        | 19/112 [00:03<00:17,  5.47it/s]

...

 99%|█████████▊| 72/73 [00:11<00:00,  5.98it/s]

Request: <module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>; Status code: 403
Request: <module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>; Status code: 403


100%|██████████| 73/73 [00:11<00:00,  6.31it/s]


Request: <module 'requests' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\requests\\__init__.py'>; Status code: 403


KeyboardInterrupt: 

In [ ]:
df.to_csv('data_craigslist.csv')

In [ ]:
df

,level_0,index,Price,Neighborhood,URL,Date,bicycle frame material,bicycle type,frame size,suspension,...,text,condition,electric assist,make / manufacturer,model name / number,brake type,handlebar type,part type,size / dimensions,serial number
0,0,0,1900.0,"(Hudson, FL)",https://newyork.craigslist.org/brk/bik/d/hudso...,2022-04-14 10:05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,300.0,(Fairfield CT),https://newyork.craigslist.org/fct/bik/d/fairf...,2022-04-14 10:05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,190.0,(BROOKLYN),https://newyork.craigslist.org/brk/bik/d/brook...,2022-04-14 10:01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,250.0,(West Village),https://newyork.craigslist.org/mnh/bik/d/new-y...,2022-04-14 08:25,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,800.0,(Brooklyn),https://newyork.craigslist.org/brk/bik/d/brook...,2022-04-14 00:22,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,2933,3113,999.0,(Financial District),https://newyork.craigslist.org/mnh/bik/d/new-y...,2022-03-31 20:33,aluminum,bmx,ALL,NaN,...,"The PK Ripper was the raddest, baddest, and mo...",new,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2934,2934,3114,3850.0,(brooklyn),https://newyork.craigslist.org/brk/bik/d/brook...,2022-03-31 15:54,steel,gravel,XL,NaN,...,I ordered this bike back in NOV.2020 with all ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2935,2935,3115,125.0,(Garden City),https://newyork.craigslist.org/lgi/bik/d/new-h...,2022-03-31 11:50,other/unknown,bmx,Medium,NaN,...,Haro F3 BMX-$175 Mongoose Racer X BMX-$175 T...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2936,2936,3116,190.0,(BROOKLYN),https://newyork.craigslist.org/brk/bik/d/brook...,2022-03-31 11:35,other/unknown,track,54,NaN,...,Pick up in Brooklyn 11229 show contact info N...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
